AIM : compute the time series of the decomposition of the horizontal advection 

In [2]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr

from dask.distributed import Client
c = Client()
# c = Client(n_workers=6, threads_per_worker=1, memory_limit="8 GiB",local_directory="/gpfsscratch/rech/cli/uor98hu/dask_tmp/")
import lib_SLXtools_IMHOTEP_2021 as slx
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 20.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38179,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 20.00 GiB
Comm: tcp://127.0.0.1:37011,Total threads: 2
Dashboard: http://127.0.0.1:39445/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:41647,


In [3]:
%%time
# -------------------
# loading data
# -------------------

y1 = 1980 # start year
y2 = 2018 # end year
# chunk_size = {'time_counter':1,'deptht':1} 
chunk_size = {"x":349,"y":500,'time_counter':1,'deptht':40} 

# read mask
# maskBG = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/maskBG.nc', chunks=chunk_size).maskBG
maskBG = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc', chunks={"x":349,"y":500}).BGRmask.squeeze()

# read scale factors
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr=xr.open_dataset(diri+'mesh_hgr.nc',chunks={"x":349,"y":500}).squeeze()
e1t = mesh_hgr.e1t.where(maskBG,drop=True)
e2t = mesh_hgr.e2t.where(maskBG,drop=True)

# read e3t
prefix = "eORCA025.L75-IMHOTEP"
diridatref="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+".GAI-S/"

# years to read data from:
T_dsbig = xr.open_mfdataset(diridatref+"1m/"+str(y1)+"/*1m_gridT.nc",decode_coords=False,chunks=chunk_size,parallel=True).e3t.where(maskBG,drop=True)

for year in np.arange(y1+1,y2+1):
    # grid T
    listTfiles=diridatref+"1m/"+str(year)+"/*1m_gridT.nc"
    T_ds = xr.open_mfdataset(listTfiles,decode_coords=False,chunks=chunk_size,parallel=True).e3t.where(maskBG,drop=True)
    T_dsbig = xr.combine_by_coords([T_dsbig, T_ds],combine_attrs="drop")
    
e3t = T_dsbig.e3t

# gridcell volume
btbox = e3t*e2t*e1t

# read data to compare to 
# AIadv_tot = xr.open_dataset("/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/averageAI_Sbudgetadv_h_v1980_2018_1m_000_bot_maskBG.nc",chunks=chunk_size).advh_boxtscomp
# GAIadv_tot = xr.open_dataset("/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/averageGAI_Sbudgetadv_h_v1980_2018_1m_000_bot_maskBG.nc",chunks=chunk_size).advh_boxtscomp

# read adv_h decomposed budget terms
# .sel(time_counter=slice(y1,y2))
# dirisc= "/gpfswork/rech/cli/uor98hu/MYDATA/temp/advectionterms/advtermsGR/"

# GAI -AI
# dirisc= "/gpfsscratch/rech/cli/uor98hu/BILANS/1dDecomposition/advterms/" 
# VeloGrSGr  = xr.open_mfdataset(dirisc +"VeloGrSGr/"+"*VeloGrSGr.nc" ,combine="nested", concat_dim=["time_counter"], chunks=chunk_size, parallel = True).VeloGrSGr.where(maskBG,drop=True)
# VeloGrSAI  = xr.open_mfdataset(dirisc +"VeloGrSAI/"+"*VeloGrSAI.nc" ,combine="nested", concat_dim=["time_counter"], chunks=chunk_size, parallel = True).VeloGrSAI.where(maskBG,drop=True)
# VeloAISGr  = xr.open_mfdataset(dirisc +"VeloAISGr/"+"*VeloAISGr.nc" ,combine="nested", concat_dim=["time_counter"], chunks=chunk_size, parallel = True).VeloAISGr.where(maskBG,drop=True)

# S- AI
dirisc= "/gpfsscratch/rech/cli/uor98hu/BILANS/1dDecomposition/RIVERS/advterms/" 
VeloRivSRiv  = xr.open_mfdataset(dirisc +"VeloRivSRiv/"+"*VeloRivSRiv.nc" ,combine="nested", concat_dim=["time_counter"], chunks=chunk_size, parallel = True).VeloRivSRiv.where(maskBG,drop=True)
VeloRivSAI  = xr.open_mfdataset(dirisc +"VeloRivSAI/"+"*VeloRivSAI.nc" ,combine="nested", concat_dim=["time_counter"], chunks=chunk_size, parallel = True).VeloRivSAI.where(maskBG,drop=True)
VeloAISRiv  = xr.open_mfdataset(dirisc +"VeloAISRiv/"+"*VeloAISRiv.nc" ,combine="nested", concat_dim=["time_counter"], chunks=chunk_size, parallel = True).VeloAISRiv.where(maskBG,drop=True)


CPU times: user 23.1 s, sys: 2.32 s, total: 25.4 s
Wall time: 54.3 s


In [3]:
# # GAI - AI
# VeloGrSGr = VeloGrSGr.assign_coords(time_counter=e3t.time_counter)
# VeloGrSAI = VeloGrSAI.assign_coords(time_counter=e3t.time_counter)
# VeloAISGr = VeloAISGr.assign_coords(time_counter=e3t.time_counter)

In [4]:
# S - AI
VeloRivSRiv = VeloRivSRiv.assign_coords(time_counter=e3t.time_counter)
VeloRivSAI = VeloRivSAI.assign_coords(time_counter=e3t.time_counter)
VeloAISRiv = VeloAISRiv.assign_coords(time_counter=e3t.time_counter)

## for whole water column

In [7]:
%%time
voltotal = btbox.sum(("deptht","y","x")).compute()

CPU times: user 26.1 s, sys: 1.26 s, total: 27.4 s
Wall time: 1min 48s


In [5]:
%%time

# GAI -AI
VeloGrSGr_boxts  = (VeloGrSGr*btbox).sum(("deptht","y","x")) / voltotal
VeloGrSAI_boxts  = (VeloGrSAI*btbox).sum(("deptht","y","x")) / voltotal
VeloAISGr_boxts  = (VeloAISGr*btbox).sum(("deptht","y","x")) / voltotal

VeloGrSGr_boxtscomp = VeloGrSGr_boxts.compute()
VeloGrSAI_boxtscomp = VeloGrSAI_boxts.compute()
VeloAISGr_boxtscomp = VeloAISGr_boxts.compute()

# retirer la time mean de toutes les séries temporelles 
VeloGrSGr_boxtscomp = VeloGrSGr_boxtscomp - VeloGrSGr_boxtscomp.mean()
VeloGrSAI_boxtscomp = VeloGrSAI_boxtscomp - VeloGrSAI_boxtscomp.mean()
VeloAISGr_boxtscomp = VeloAISGr_boxtscomp - VeloAISGr_boxtscomp.mean()


CPU times: user 3min 8s, sys: 10.7 s, total: 3min 19s
Wall time: 22min 35s


In [6]:
# adv_tot_diff_1d = VeloAISGr_boxtscomp + VeloGrSAI_boxtscomp + VeloGrSGr_boxtscomp

In [8]:
%%time

# S -AI
VeloRivSRiv_boxts  = (VeloRivSRiv*btbox).sum(("deptht","y","x")) / voltotal
VeloRivSAI_boxts  = (VeloRivSAI*btbox).sum(("deptht","y","x")) / voltotal
VeloAISRiv_boxts  = (VeloAISRiv*btbox).sum(("deptht","y","x")) / voltotal

VeloRivSRiv_boxtscomp = VeloRivSRiv_boxts.compute()
VeloRivSAI_boxtscomp = VeloRivSAI_boxts.compute()
VeloAISRiv_boxtscomp = VeloAISRiv_boxts.compute()

# retirer la time mean de toutes les séries temporelles 
VeloRivSRiv_boxtscomp = VeloRivSRiv_boxtscomp - VeloRivSRiv_boxtscomp.mean()
VeloRivSAI_boxtscomp = VeloRivSAI_boxtscomp - VeloRivSAI_boxtscomp.mean()
VeloAISRiv_boxtscomp = VeloAISRiv_boxtscomp - VeloAISRiv_boxtscomp.mean()


CPU times: user 2min 44s, sys: 6.24 s, total: 2min 51s
Wall time: 8min 29s


In [7]:
# # GAI -AI

# # sauvegarder les timeseries calculées
# ds = xr.Dataset(
#     data_vars=dict(
#         VeloGrSGr_boxtscomp=(["time_counter"], VeloGrSGr_boxtscomp.data),
#         VeloGrSAI_boxtscomp=(["time_counter"], VeloGrSAI_boxtscomp.data),
#         VeloAISGr_boxtscomp=(["time_counter"], VeloAISGr_boxtscomp.data),

#     ),

#     coords=dict(
#         time_counter=VeloGrSGr_boxtscomp.time_counter.values,
#     ),

#     attrs=dict(description="time series 1980-2018 of BGRmask spatial average of horizontal decomposition of salinity budget terms in 10-3/s from surface to bottom for GAI - AI"),

# )
# ds.to_netcdf(path='/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/'+'average_adv1dDecompostion_Sbudgetterms'+str(y1)+'_'+str(y2)+'_1m_000_bottom_BGRmaskobs.nc', mode='w')

In [9]:
# S -AI

# sauvegarder les timeseries calculées
ds = xr.Dataset(
    data_vars=dict(
        VeloRivSRiv_boxtscomp=(["time_counter"], VeloRivSRiv_boxtscomp.data),
        VeloRivSAI_boxtscomp=(["time_counter"], VeloRivSAI_boxtscomp.data),
        VeloAISRiv_boxtscomp=(["time_counter"], VeloAISRiv_boxtscomp.data),

    ),

    coords=dict(
        time_counter=VeloRivSRiv_boxtscomp.time_counter.values,
    ),

    attrs=dict(description="time series 1980-2018 of BGRmask spatial average of horizontal decomposition of salinity budget terms in 10-3/s from surface to bottom for S - AI "),

)
ds.to_netcdf(path='/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/'+'average_adv1dDecompostion_Sbudgetterms'+str(y1)+'_'+str(y2)+'_1m_000_bottom_BGRmaskobsS_AI.nc', mode='w')

## for 0-300 m

In [8]:
# %%time

# ### GAI- AI

# zend = 35
# voltotal = btbox.isel(deptht=np.arange(zend)).sum(("deptht","y","x")).compute()

# VeloGrSGr_boxts  = (VeloGrSGr*btbox).isel(deptht=np.arange(zend)).sum(("deptht","y","x")) / voltotal
# VeloGrSAI_boxts  = (VeloGrSAI*btbox).isel(deptht=np.arange(zend)).sum(("deptht","y","x")) / voltotal
# VeloAISGr_boxts  = (VeloAISGr*btbox).isel(deptht=np.arange(zend)).sum(("deptht","y","x")) / voltotal

# VeloGrSGr_boxtscomp = VeloGrSGr_boxts.compute()
# VeloGrSAI_boxtscomp = VeloGrSAI_boxts.compute()
# VeloAISGr_boxtscomp = VeloAISGr_boxts.compute()

# # retirer la time mean de toutes les séries temporelles 
# VeloGrSGr_boxtscomp = VeloGrSGr_boxtscomp - VeloGrSGr_boxtscomp.mean()
# VeloGrSAI_boxtscomp = VeloGrSAI_boxtscomp - VeloGrSAI_boxtscomp.mean()
# VeloAISGr_boxtscomp = VeloAISGr_boxtscomp - VeloAISGr_boxtscomp.mean()

# # sauvegarder les timeseries calculées
# ds = xr.Dataset(
#     data_vars=dict(
#         VeloGrSGr_boxtscomp=(["time_counter"], VeloGrSGr_boxtscomp.data),
#         VeloGrSAI_boxtscomp=(["time_counter"], VeloGrSAI_boxtscomp.data),
#         VeloAISGr_boxtscomp=(["time_counter"], VeloAISGr_boxtscomp.data),

#     ),

#     coords=dict(
#         time_counter=VeloGrSGr_boxtscomp.time_counter.values,
#     ),

#     attrs=dict(description="time series 1980-2018 of BGRmask spatial average of horizontal decomposition of salinity budget terms in 10-3/s from surface to 300 m deep for GAI-AI"),

# )
# ds.to_netcdf(path='/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/'+'average_adv1dDecompostion_Sbudgetterms'+str(y1)+'_'+str(y2)+'_1m_000_300_BGRmaskobsGAI_AI.nc', mode='w')

In [17]:
%%time

zend = 35
voltotal = btbox.isel(deptht=np.arange(zend)).sum(("deptht","y","x")).compute()

VeloRivSRiv_boxts  = (VeloRivSRiv*btbox).isel(deptht=np.arange(zend)).sum(("deptht","y","x")) / voltotal
VeloRivSAI_boxts  = (VeloRivSAI*btbox).isel(deptht=np.arange(zend)).sum(("deptht","y","x")) / voltotal
VeloAISRiv_boxts  = (VeloAISRiv*btbox).isel(deptht=np.arange(zend)).sum(("deptht","y","x")) / voltotal

VeloRivSRiv_boxtscomp = VeloRivSRiv_boxts.compute()
VeloRivSAI_boxtscomp = VeloRivSAI_boxts.compute()
VeloAISRiv_boxtscomp = VeloAISRiv_boxts.compute()

# retirer la time mean de toutes les séries temporelles 
VeloRivSRiv_boxtscomp = VeloRivSRiv_boxtscomp - VeloRivSRiv_boxtscomp.mean()
VeloRivSAI_boxtscomp = VeloRivSAI_boxtscomp - VeloRivSAI_boxtscomp.mean()
VeloAISRiv_boxtscomp = VeloAISRiv_boxtscomp - VeloAISRiv_boxtscomp.mean()


CPU times: user 2min 10s, sys: 9.29 s, total: 2min 19s
Wall time: 5min 4s


In [18]:
# sauvegarder les timeseries calculées
ds = xr.Dataset(
    data_vars=dict(
        VeloRivSRiv_boxtscomp=(["time_counter"], VeloRivSRiv_boxtscomp.data),
        VeloRivSAI_boxtscomp=(["time_counter"], VeloRivSAI_boxtscomp.data),
        VeloAISRiv_boxtscomp=(["time_counter"], VeloAISRiv_boxtscomp.data),

    ),

    coords=dict(
        time_counter=VeloRivSRiv_boxtscomp.time_counter.values,
    ),

    attrs=dict(description="time series 1980-2018 of BGRmask spatial average of horizontal decomposition of salinity budget terms in 10-3/s from surface to 300 m deep for S-AI"),

)
ds.to_netcdf(path='/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/'+'average_adv1dDecompostion_Sbudgetterms'+str(y1)+'_'+str(y2)+'_1m_000_300_BGRmaskobsS_AI.nc', mode='w')